# HTTP wrapper for kachaka-api

kachaka-apiライブラリをhttp経由でアクセスできるようにするデモです  

使用例:
```
curl <カチャカのIPアドレス>:26502/kachaka/robot_serial_number
curl <カチャカのIPアドレス>:26502/kachaka/auto_homing_enabled
curl <カチャカのIPアドレス>:26502/kachaka/auto_homing_enabled --json '{"enable": false}'
curl <カチャカのIPアドレス>:26502/kachaka/move_shelf --json '{"shelf_name_or_id":"シェルフ", "location_name_or_id":"キッチン"}'
curl <カチャカのIPアドレス>:26502/kachaka/speak --json '{"text":"APIから指令を受信しました"}'
curl <カチャカのIPアドレス>:26502/kachaka/return_home
```

In [ ]:
%run setup_demo_env.py

依存ライブラリのインストールを行います。初回インストール後にkernelの再起動が必要です

In [ ]:
!pip3 install -q fastapi uvicorn

In [ ]:
from collections.abc import Iterable

import kachaka_api
from fastapi import FastAPI, HTTPException
from google._upb._message import RepeatedCompositeContainer
from google.protobuf.json_format import MessageToDict

app = FastAPI()
kachaka_client = kachaka_api.aio.KachakaApiClient("192.168.1.202:26400")


@app.on_event("startup")
async def init_channel() -> None:
    await kachaka_client.update_resolver()


def to_dict(response):
    if response.__class__.__module__ == "kachaka_api_pb2":
        return MessageToDict(response)
    if (
        isinstance(response, tuple)
        or isinstance(response, list)
        or isinstance(response, RepeatedCompositeContainer)
    ):
        ret = []
        for r in response:
            ret.append(to_dict(r))
        return ret
    return response


def get_method_or_404(attributes):
    for attr in attributes:
        if hasattr(kachaka_client, attr):
            return getattr(kachaka_client, attr)
    raise HTTPException(status_code=404, detail="Method not found")


@app.get("/kachaka/{item:path}")
async def get(item: str):
    method = get_method_or_404(["get_" + item, item])
    response = await method()
    return to_dict(response)


@app.post("/kachaka/{item:path}")
async def post(item: str, params: dict):
    method = get_method_or_404([item, "set_" + item])
    response = await method(**params)
    return to_dict(response)

In [ ]:
import uvicorn

config = uvicorn.Config(app)
config.host = "0.0.0.0"
config.port = 26502
server = uvicorn.Server(config)
await server.serve()